Outstanding Questions:
1. What is more of a priority - even workload or applications being reviewed by SysCog and CellMolec person? For overflow do you want it random or to give some faculty as more acceptable in the other category than others??

To-Do:
1. Add optimization for round 2
2. Remove Score of 1 in generating scorecards (for demonstration purposes only)

# NSGP Admissions

This code assigns reviewers, generates, and merges score cards for two rounds of review. 

Assignment rules:
1. At least one sys/cog/comp and cell/molec person per applicant.
2. No applications only reviewed by student reps
3. Total workload should be evenly distributed (with "extra" applications going to faculty first before student reps), and distributed as evenly as possible within application pools. 

## Section 0
Import required packages and define functions and variables used across all sections. Always run all cells in this section before running any other sections.

In [1]:
import pandas as pd
import numpy as np
import random
import xlsxwriter
import time
import os

sep = '\\' #Define as per your operating system path seperator 

In [13]:
#Define reviewer groups
rev_groups = {'StudentReps':['Amin', 'Lambert'],
              'SysCog':['Nassar','Chirila','Truccolo','Sheinberg','Berson','Paradiso','Moore'],
              'WildCard':['Amin','Lambert'], #Contains reviewers from larger category that could be used as reviewers for smaller category
              'CellMolec':['Jaworski','Fallon','Abdelfattah','Amin','Lambert','Aizenman','Mayoral', 'Wilson']}
rev_groups['Reviewers'] = sorted(list(set([item for sublist in rev_groups.values() for item in sublist]))) #Unique list of reviewers

reviewer_colorspace = ['#F28FD4','#ffa3a2','#fcf6e7','#ff8e58','#da6175','#B05199','#446AB2',
                       '#3296A6','#24b8a0','#8ee6a1','#cae63c','#edd54f','#6bae36','#7da497','#c8cdbb']

In [3]:
#TO-DO: If uneven number of reviewers, smaller group should get extra applications first
def calc_reviewer_workload(num_applicants,rev_groups,n):
    # Calculate TOTAL workload for each reviewer across all applicants
    # Args:
    #    applicants: complete list of applicant names 
    #    rev_groups: dictionary categorizing reviewers into 'StudentReps', 'SysCog', 'CellMolec','Reviewers'
    #    n: single value, number of reviewers assigned to each application
    #
    # Returns:
    #    A dictionary containing TOTAL workload
    reviewers = rev_groups['Reviewers'] #Unique list of reviewers
    
    num_reviewers = len(reviewers)
    #num_applicants = len(applicants)
    num_reviews = num_applicants*n
    
    base_reviews = num_reviews // num_reviewers #// gives floor division, will always be integer rounded down
    remainder = num_reviews - (base_reviews*num_reviewers) # number of extra applications
  
    workload_counts = {rev: base_reviews for rev in reviewers}

    #Add remainder reviews to random subset of faculty reviewers (never student reviewers)
    fac_reviewers = list(set.difference(set(rev_groups['Reviewers']),set(rev_groups['StudentReps'])))
    random.shuffle(fac_reviewers)
    if remainder != 0:
        for i, reviewer in enumerate(fac_reviewers):
            if i<remainder:
                workload_counts[reviewer] += 1 #for remainder faculty, their workload is 1 extra
    
    if remainder > len(fac_reviewers): #if remainder is more than number of faculty, pick random subset of reps to give extra reviews to
        student_reviewers = random.sample(rev_groups['StudentReps'],remainder-len(fac_reviewers))
        for student in student_reviewers:
            workload_counts[reviewer] += 1
        
    return workload_counts

In [10]:
def assign_reviewers_multipool3(applicant_pools,rev_groups,n,min_rep=1):
    ''''
    Args:
        applicant_pools: dictionary containing list of applicant names in each pool key
        rev_groups: dictionary categorizing reviewers into 'StudentReps', 'SysCog', 'CellMolec','Reviewers'
        n: single value, number of reviewers assigned to each application
        min_rep: single value, minimum number of reviewers that should be represented from each reviewer group per application (default is 1)
    
    Returns:
        assignment_list_df: dataframe of each applicant and list of their reviewers
        reviewer_workload_df: dataframe of each reviewer and list of their assigned applicants
    ''''

    if len(rev_groups['CellMolec']) == len(rev_groups('SysCog')):
        raise ValueError('Warning! You have an even number of reviewers in groups. Use assign_reviewers_multipool.')
    elif len(rev_groups['CellMolec']) > len(rev_groups['SysCog']):       
        small_key = 'SysCog'
        big_key = 'CellMolec'
    elif len(rev_groups['SysCog'] > rev_groups['CellMolec']):
        small_key = 'CellMolec'
        big_key = 'SysCog'
       
    reviewers = rev_groups['Reviewers'] #Unique list of reviewers
    pools = list(applicant_pools.keys())
    time_limit = 1 #Time limint on attempting redraws


    SMALL_POOL = rev_groups[small_key]
    LARGE_POOL = rev_groups[big_key]
    WILDCARD_POOL = rev_groups['WildCard']

    
    #Calculate workload, keeping total evenly distributed and with pool evenly distributed (ish)
    num_applicants = sum(len(value) for value in applicant_pools.values()) #Calculate total number of applicants
    workload_counts_total = calc_reviewer_workload(num_applicants,rev_groups,n) #Total reviews per reviewer   
    
    workload = {}
    assignment_list = {}
    for pool in pools[:-1]: #Calculate workloads independently for each pool except final pool
        workload[pool] = calc_reviewer_workload(len(applicant_pools[pool]),rev_groups,n)
            
    workload_pool_total = {}
    for inner_dict in workload.values(): #sum of workloads across pools, exculding final pool
        for key, value in inner_dict.items():
            workload_pool_total[key] = workload_pool_total.get(key,0) + value
       
    workload[pools[-1]] = {key: workload_counts_total[key] - workload_pool_total[key] for key in workload_counts_total} 
    
    
    for pool in applicant_pools:
        print(f"\n--{pool} applicant pool--")
        
        workload_counts = workload[pool]
        applicants = applicant_pools[pool]                
        assignment_list[pool] = {applicant:[] for applicant in applicants}  #Populate empty assignment dictionary
        
        # --------------------------------------------------------------------------
        ## 1. PHASE 1: Global Min-Rep for Small Category (C1)
    
        print(f"Phase 1: Assigning {min_rep} from {small_key} (Small Category)")
        
        for _ in range(min_rep): # Iterate for each min_rep slot required
            
            # Shuffle applicants to ensure randomness across the assignment order
            random.shuffle(applicants)
            
            for applicant in applicants:
                
                # 1a. Attempt to find reviewer in the Small Category
                available_pool = [
                    reviewer for reviewer, count in workload_counts.items()
                    if count > 0 and reviewer not in assignment_list[pool][applicant] and reviewer in SMALL_POOL
                ]
    
                if not available_pool:
                    
                    # 1b. Fallback to WildCard Pool
                    available_pool = [
                        reviewer for reviewer, count in workload_counts.items()
                        if count > 0 and reviewer not in assignment_list[pool][applicant] and reviewer in WILDCARD_POOL
                    ]
                    
                # Assign one reviewer (if available)
                if available_pool:
                    available_weights = [workload_counts[candidate] for candidate in available_pool]
                    probabilities = [weight/sum(available_weights) for weight in available_weights]
                    chosen_reviewer = np.random.choice(available_pool, 1, replace=False, p=probabilities)[0]
        
                    assignment_list[pool][applicant].append(chosen_reviewer)
                    workload_counts[chosen_reviewer] -= 1
                else:
                    raise ValueError(f"Error: Could not find a unqiue {small_key} reviewer.")
    
        
        # --------------------------------------------------------------------------
        ## 2. PHASE 2: Global Min-Rep for Large Category (C2)
        
        print(f"Phase 2: Assigning {min_rep} from {big_key} (Large Category)")
        
        for i in range(min_rep): # Iterate for each min_rep slot required
            
            random.shuffle(applicants)
            
            for applicant in applicants:
                
                # Only assign if this application still needs C2 representation
                # (It's possible a C2 reviewer was already assigned in Phase 1 if C2 was the WildCard source)
                
                # Available reviewers in the Large Category
                available_pool = [
                    reviewer for reviewer, count in workload_counts.items()
                    if count > 0 and reviewer not in assignment_list[pool][applicant] and reviewer in LARGE_POOL
                ]
                
                # Assign one reviewer
                if available_pool:
                    available_weights = [workload_counts[candidate] for candidate in available_pool]
                    probabilities = [weight/sum(available_weights) for weight in available_weights]
                    
                    chosen_reviewer = np.random.choice(available_pool, 1, replace=False, p=probabilities)[0]
    
                   #If this is the final draw, check full assigned reveiwers. Will only catch a few because logic is bad. Main catch is in the wrapper.
                    #TO-DO: Have it break and start full selection over if it's impossible instead of getting stuck
                    if n == 2*min_rep and i == (min_rep-1): 
                        start_time = time.time()
                        proposed_final_reviewers = assignment_list[pool][applicant] + [chosen_reviewer]
                        while sorted(proposed_final_reviewers) == sorted(rev_groups['StudentReps']):
                            chosen_reviewer = np.random.choice(available_pool, 1, replace=False, p=probabilities)[0]
                            current_time = time.time()
                            if current_time - start_time >= time_limit:
                                print("Break Rule 1: Student only reviewers")
                                break                        
                
                    assignment_list[pool][applicant].append(chosen_reviewer)
                    workload_counts[chosen_reviewer] -= 1
                else:
                    raise ValueError(f"Error: Not enough {big_key} reviewers. You did some math wrong somewhere...")
     
       
        
        # --------------------------------------------------------------------------
        ## 3. PHASE 3: Global Assignment for Remaining Slots (Completion to n)  
        
        if n > 2*min_rep:
            print(f"\n--- Phase 3: Filling Remaining {n-2*min_rep} Slots (Total {n} Reviews) ---")
            
            for i in range(n-2*min_rep):
                random.shuffle(applicants)
                for applicant in applicants:                    
                    # Full Pool: All reviewers with remaining workload, excluding assigned ones
                    available_pool = [
                        reviewer for reviewer, count in workload_counts.items()
                        if count > 0 and reviewer not in assignment_list[pool][applicant] 
                    ]
            
                    # Assign one reviewer at a time until the slot is filled or pool is exhausted
                    if available_pool:
                        available_weights = [workload_counts[candidate] for candidate in available_pool]
                        probabilities = [weight/sum(available_weights) for weight in available_weights]
                        chosen_reviewer = np.random.choice(available_pool, 1, replace=False, p=probabilities)[0]
                        #TO-DO: Need to put in a check if there are > 2 assignments and >2 student reps (eg 3 reps and assignment of 3)
                            
                                    
                        assignment_list[pool][applicant].append(chosen_reviewer)
                        workload_counts[chosen_reviewer] -= 1
                    else:
                        raise ValueError(f'Not enough unique reviewers for {applicant}')
    
    merged_assignment_list = {key:value for sub_dict in assignment_list.values() for key,value in sub_dict.items()}
    reviewer_workload = {reviewer:[] for reviewer in reviewers}   
    for app, revs in merged_assignment_list.items():
        for rev in revs:
            reviewer_workload[rev].append(app)
    
    # total = 0
    # for key,value in reviewer_workload.items():
    #     total = total + len(value)
    #     print(f"Number of apps for {key} is {len(value)}")
    # print(f"Total reviews is {total}")
        
    reviewer_workload_df = pd.DataFrame(reviewer_workload.items(),columns = ['Reviewer','Applicants'])
    assignment_list_df = pd.DataFrame(merged_assignment_list.items(),columns = ['Applicant','Reviewers'])
            
    return assignment_list_df, reviewer_workload_df

In [5]:

def assign_reviewers_multipool(applicant_pools,rev_groups,n):
    '''
    This function works to assign reviewers to applications if you have an equal number of reviewers in each required category.
    
    Args:
        applicant_pools: dictionary containing list of applicant names in each pool key
        rev_groups: dictionary categorizing reviewers into 'StudentReps', 'SysCog', 'CellMolec','Reviewers'
        n: single value, number of reviewers assigned to each application
    
    Returns:
        assignment_list_df: dataframe of each applicant and list of their reviewers
        reviewer_workload_df: dataframe of each reviewer and list of their assigned applicants
    '''

    if len(rev_groups['CellMolec']) != len(rev_groups('SysCog')):
        raise ValueError('Warning! You have an unequal number of reviewers in groups. Use assign_reviewers_multipool2.')
        
    reviewers = rev_groups['Reviewers'] #Unique list of reviewers
    pools = list(applicant_pools.keys())

    #Calculate workload, keeping total evenly distributed and with pool evenly distributed (ish)
    num_applicants = sum(len(value) for value in applicant_pools.values()) #Calculate total number of applicants
    workload_counts_total = calc_reviewer_workload(num_applicants,rev_groups,n) #Total reviews per reviewer   
    
    workload = {}
    for pool in pools[:-1]: #Calculate workloads independently for each pool except final pool
        workload[pool] = calc_reviewer_workload(len(applicant_pools[pool]),rev_groups,n)
            
    workload_pool_total = {}
    for inner_dict in workload.values(): #sum of workloads across pools, exculding final pool
        for key, value in inner_dict.items():
            workload_pool_total[key] = workload_pool_total.get(key,0) + value
       
    workload[pools[-1]] = {key: workload_counts_total[key] - workload_pool_total[key] for key in workload_counts_total}


    assignment_list = {}

    for pool in applicant_pools:
        workload_counts = workload[pool]
        applicants = applicant_pools[pool]
       
        #Create assignment list
        random.shuffle(applicants)
        assignment_list[pool] = {applicant:[] for applicant in applicants}  
        for applicant in applicants:
            #Pull list of possible reviewers based on if their workload is already full or not
            available_reviewers = [candidates for candidates, count in workload_counts.items() if count > 0]
            if len(available_reviewers) < n:
                raise ValueError(
                    f"Cannot find {n} unique reviewers for '{applicant}'. The remaining pool "
                    f"of reviewers with available slots ({len(available_reviewers)}) is too small."
                )
                

            #Check if possible to select subset from available reviewers that will not break rules, else raise ValueError
            #From available reviewers, choose n at random    
            if sorted(available_reviewers) == sorted(rev_groups['StudentReps']):
                raise ValueError('Break Rule 1: Student only reviewers')
            if not any(item in available_reviewers for item in rev_groups['SysCog']) and any(item in available_reviewers for item in rev_groups['CellMolec']):
                raise ValueError('Break Rule 2: Distribution of fields')
            #Draw random set of reviewers from those with workload available
            available_weights = [count for candidates, count in workload_counts.items() if count > 0]
            probabilities = [weight/sum(available_weights) for weight in available_weights]
            #chosen_reviewers = random.sample(available_reviewers, n)
            chosen_reviewers = np.random.choice(available_reviewers, n, replace = False,p=probabilities)  
            
            #Check reviewers chosen follows rule, or else pick another sample
            start_time = time.time()
            time_limit = 1
            while sorted(chosen_reviewers) == sorted(rev_groups['StudentReps']) or not (any(item in chosen_reviewers for item in rev_groups['SysCog']) and any(item in chosen_reviewers for item in rev_groups['CellMolec'])):
                #chosen_reviewers = random.sample(available_reviewers, n) 
                chosen_reviewers = np.random.choice(available_reviewers, n, replace = False,p=probabilities)  
    
                #Sometimes code gets hung. Introduce timeout to restart.
                current_time = time.time()
                if current_time - start_time >= time_limit:
                    print("Assign Reviewers Timeout")
                    break
    
                
            assignment_list[pool][applicant] = chosen_reviewers
            for reviewer in chosen_reviewers:
                workload_counts[reviewer] -= 1

    merged_assignment_list = {key:value for sub_dict in assignment_list.values() for key,value in sub_dict.items()}

    #Reverse map assignments
    reviewer_workload = {reviewer:[] for reviewer in reviewers}   
    for app, revs in merged_assignment_list.items():
        for rev in revs:
            reviewer_workload[rev].append(app)

    total = 0
    for key,value in reviewer_workload.items():
        total = total + len(value)
        print(f"Number of apps for {key} is {len(value)}")
    print(f"Total reviews is {total}")
        
    reviewer_workload_df = pd.DataFrame(reviewer_workload.items(),columns = ['Reviewer','Applicants'])
    assignment_list_df = pd.DataFrame(merged_assignment_list.items(),columns = ['Applicant','Reviewers'])
            
    return assignment_list_df, reviewer_workload_df

# ROUND 1

## Section 1A: Import Data (Round 1)

In [8]:
review_round = 1
n_rev_per_app = 2 #number of reviewers assigned to each application
directory_path = f"Round {review_round}"
print(f"Directory path = {directory_path}")
os.makedirs(directory_path, exist_ok=True)

applicant_info = pd.read_excel('General Submitted Applications Query 20241202-080538.xlsx') #Read Data Spreadsheet exported from Slate


# All applicants in single pool
applicants = applicant_info['Person Name'].values

# Separate TG-Eligible and International applicant pools
applicant_pools = {'International':applicant_info.loc[applicant_info['Citizenship Status']=='Foreign Citizen','Person Name'].values,
                  'TG-Eligible':applicant_info.loc[applicant_info['Citizenship Status']!='Foreign Citizen','Person Name'].values}

print(f"\n--Imported Data for Round {review_round}, {len(applicant_info)} applications to review--")


Directory path = Round 1
386
--Imported Data for Round 1, 386 applications to review--


## Section 1B: Assign Reviewers

Randomly assign reviewers to applications based on the following rules:
1. Workload should be distributed evenly
2. Application assignments should be as random as possible (two reviewers should not have identical pools of applications)
3. An individual application should never be reviewed by ONLY the student reps
4. Application should be reviewed by a sys/cog/comp and cell/molec person


In [11]:
#Run until it finds proper assignment with no errors. 
start_time = time.time()
timeout_seconds = 30
target_tuple = tuple(sorted(rev_groups['StudentReps']))
assignment_successful = False

while True:
    
    if time.time() - start_time >= timeout_seconds:
        print("Timeout reached, try again")
        break
    try:
        assignment_list, reviewer_workload = assign_reviewers_multipool3(applicant_pools,rev_groups,n_rev_per_app)

        df = assignment_list.copy()
        df['TupleColumn'] = df['Reviewers'].apply(tuple)
        is_present = df['TupleColumn'].isin([target_tuple]).any()
        if is_present:
            assignment_successful = True
            break
        else:
            print('Attempt failed validation: Student Reviewers only')

    except ValueError as e:
        print(f"Error: {e}")

print(f"--Assigned Reviewers for Round {review_round}--\n")
for index,row in reviewer_workload.iterrows():
    print(f"Number of apps for {row['Reviewer']} is {len(row['Applicants'])}")      

#Save assignment stats for admissions chair to review
assignment_statistics = assignment_list['Reviewer'].valuecounts()
assignment_statistics.to_excel(f"{directory_path}{sep}Round {review_round} Assignment Statistics.xlsx")



--International applicant pool--
Phase 1: Assigning 1 from SysCog (Small Category)
Phase 2: Assigning 1 from CellMolec (Large Category)
Break Rule 1: Student only reviewers

--TG-Eligible applicant pool--
Phase 1: Assigning 1 from SysCog (Small Category)
Phase 2: Assigning 1 from CellMolec (Large Category)
Break Rule 1: Student only reviewers
Attempt failed validation: Student Reviewers only

--International applicant pool--
Phase 1: Assigning 1 from SysCog (Small Category)
Phase 2: Assigning 1 from CellMolec (Large Category)
Break Rule 1: Student only reviewers
Break Rule 1: Student only reviewers

--TG-Eligible applicant pool--
Phase 1: Assigning 1 from SysCog (Small Category)
Phase 2: Assigning 1 from CellMolec (Large Category)
Break Rule 1: Student only reviewers
Break Rule 1: Student only reviewers
Attempt failed validation: Student Reviewers only

--International applicant pool--
Phase 1: Assigning 1 from SysCog (Small Category)
Phase 2: Assigning 1 from CellMolec (Large Categor

## Section 1C: Generate score cards

In [18]:
# Generate master spreadsheet for manual review and edits from the admissions committee chair.

print(f"--Generating Assignment Spreadsheet for Round {review_round}--")
full_df = applicant_info.copy()
reviewers = rev_groups['Reviewers'] 
for reviewer in reviewers:
    reviewer_df = applicant_info[applicant_info['Person Name'].isin(reviewer_workload['Applicants'].loc[reviewer_workload['Reviewer']==reviewer].values[0])]
    reviewer_df = reviewer_df.sort_values(by='Person Name')
    reviewer_df.insert(0, reviewer, 'ASSIGNED') #Add blank column for Score (position, name, value)


    num_int = len(reviewer_df.loc[reviewer_df['Citizenship Status']=='Foreign Citizen'])
    num_tg = len(reviewer_df.loc[reviewer_df['Citizenship Status']!='Foreign Citizen'])
    print(f"{reviewer} has {num_int} International and {num_tg} TG-Eligible, {num_int + num_tg} total applications")   

    full_df = pd.merge(full_df,reviewer_df[[reviewer,'Person Name']], how = 'left',on = 'Person Name')

#Reorder columns so scores are first
num_reviewers = len(reviewers)
cols = full_df.columns.tolist()
cols = cols[-num_reviewers:]+cols[:-num_reviewers]
full_df = full_df[cols]
full_df.head()

full_int = full_df.loc[full_df['Citizenship Status'] == 'Foreign Citizen']

full_tg = full_df.loc[full_df['Citizenship Status'] != 'Foreign Citizen']



print(f"--Formatting Assignment Spreadsheet for Round {review_round}--")

#Create file, separate tabs for International and TG-Eligible pools
filename = f"{directory_path}{sep}Round {review_round} Assignments.xlsx"
writer = pd.ExcelWriter(filename,engine = 'xlsxwriter')
#score_df.to_excel(writer,sheet_name = index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
full_int.to_excel(writer,sheet_name = 'International', index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
full_tg.to_excel(writer,sheet_name = 'TG-Eligible', index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format

# --Format score card headers--
workbook = writer.book
worksheet1 = writer.sheets['International']
worksheet2 = writer.sheets['TG-Eligible']

header_format = workbook.add_format({
    'bold': True,
    'font_size': 13,
    'align':'center',
    'font_color': '#ffffff',
    'fg_color':'#000000',
    'text_wrap':True
    
})
for col_num, value in enumerate(full_int.columns.values):
    worksheet1.write(0, col_num, value, header_format) #(row,column,value,format)


for col_num, value in enumerate(full_tg.columns.values):
    worksheet2.write(0, col_num, value, header_format) #(row,column,value,format)


# --Additional formatting--
first_row = 1
last_row = 300
first_col = 0
last_col = num_reviewers - 1

#center align all of the scores
format0 = workbook.add_format({'align':'center'})
worksheet1.set_column(0,last_col,10,format0)
#worksheet1.set_column(last_col+1,25,25,10)
worksheet1.autofit() #Autofit (finicky, which is why I do minimum width first)


worksheet2.set_column(0,last_col,10,format0)
#worksheet2.set_column(last_col+1,25,25,10)
worksheet2.autofit() #Autofit (finicky, which is why I do minimum width first)
                      

# --Conditional formatting--


# Color faculty scores (for prettiness! Definitely not necessary)

for idx, value in enumerate(reviewers):
    rev_format = workbook.add_format({'bg_color':reviewer_colorspace[idx],
                                     'bold':True,
                                     'align':'center'})
    worksheet1.conditional_format(first_row,idx,last_row,idx,
                                 {'type':'no_blanks',
                                 'format':rev_format})
    worksheet2.conditional_format(first_row,idx,last_row,idx,
                                 {'type':'no_blanks',
                                 'format':rev_format})
    


writer.close() #Without this line the file will be corrupt

--Generating Assignment Spreadsheet for Round 1--
Abdelfattah has 22 International and 30 TG-Eligible, 52 total applications
Aizenman has 22 International and 30 TG-Eligible, 52 total applications
Amin has 21 International and 30 TG-Eligible, 51 total applications
Berson has 22 International and 30 TG-Eligible, 52 total applications
Chirila has 22 International and 30 TG-Eligible, 52 total applications
Fallon has 22 International and 29 TG-Eligible, 51 total applications
Jaworski has 22 International and 29 TG-Eligible, 51 total applications
Lambert has 21 International and 30 TG-Eligible, 51 total applications
Mayoral has 22 International and 30 TG-Eligible, 52 total applications
Moore has 22 International and 29 TG-Eligible, 51 total applications
Nassar has 22 International and 29 TG-Eligible, 51 total applications
Paradiso has 22 International and 30 TG-Eligible, 52 total applications
Sheinberg has 22 International and 29 TG-Eligible, 51 total applications
Truccolo has 22 Internatio

In [23]:
#For score cards: Have ALL applicant info listed with column indicating which are "assigned" - separate tabs for international and tg-eligible
print(f"--Generating Scorecards for Round {review_round}--")

assignment_filename = f'Round {review_round} Assignments.xlsx'
#assignment_df = pd.read_excel(f"{directory_path}{sep}{assignment_filename}")

all_assignment_sheets = pd.read_excel(f"{directory_path}{sep}{assignment_filename}", sheet_name=None)
assignment_df = pd.DataFrame()
for sheet_name, df in all_assignment_sheets.items(): #Concatenate info from all tabs into one (all have common)
    assignment_df = pd.concat([assignment_df,df],axis = 0, ignore_index = True)

reviewers = rev_groups['Reviewers']
for reviewer in reviewers:
    assignment_is_present = assignment_df[reviewer].notna()
    reviewer_assignment_list = np.where(assignment_is_present,'ASSIGNED','')
    
    reviewer_df = assignment_df.iloc[:,len(reviewers):]
  
    reviewer_df.insert(0, 'Score', '') #Add blank column for Score (position, name, value)
    reviewer_df.insert(1, 'Notes', '') #Add blank column for Score (position, name, value)
    reviewer_df.insert(2,'Assigned',reviewer_assignment_list)  

    app_int = reviewer_df.loc[reviewer_df['Citizenship Status'] == 'Foreign Citizen']
    app_tg = reviewer_df.loc[reviewer_df['Citizenship Status'] != 'Foreign Citizen']

    app_int_sort = app_int.sort_values(by=['Assigned','Person Name'],ascending=[False, True])
    app_tg_sort = app_tg.sort_values(by=['Assigned','Person Name'],ascending=[False, True])
    
    #reviewer_df = scorecard_base_df[scorecard_base_df['Reviewers'].isin([reviewer])].copy()
    
    #reviewer_df = reviewer_df.drop(columns = 'Reviewers')
    reviewer_df = reviewer_df.sort_values(by='Person Name')

    filename = f"{directory_path}{sep}{reviewer}_Scorecard - Round {review_round}.xlsx"
    writer = pd.ExcelWriter(filename,engine = 'xlsxwriter')
    app_int_sort.to_excel(writer,sheet_name = 'International', index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
    app_tg_sort.to_excel(writer,sheet_name = 'TG-Eligible', index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
    
    #Format score card
    workbook = writer.book
    worksheet1 = writer.sheets['International']
    worksheet2 = writer.sheets['TG-Eligible']
    header_format = workbook.add_format({
        'bold': True,
        'font_size': 13,
        'align':'center',
        'font_color': '#ffffff',
        'fg_color':'#000000'       
        
    })
    for col_num, value in enumerate(reviewer_df.columns.values):
        worksheet1.write(0, col_num, value, header_format) #(row,column,value,format)
        worksheet2.write(0, col_num, value, header_format) #(row,column,value,format)
        
    worksheet1.set_column(0,25,15) #Set minimum column width
    worksheet1.autofit() #Autofit (finicky, which is why I do minimum width first)
    
    worksheet2.set_column(0,25,15) #Set minimum column width
    worksheet2.autofit() #Autofit (finicky, which is why I do minimum width first)
    
    writer.close() #Without this line the file will be corrupt

    num_int = len(reviewer_df.loc[reviewer_df['Citizenship Status']=='Foreign Citizen'])
    num_tg = len(reviewer_df.loc[reviewer_df['Citizenship Status']!='Foreign Citizen'])
    print(f"{reviewer} has {num_int} International and {num_tg} TG-Eligible, {num_int + num_tg} total applications")    
    
    
    

--Generating Scorecards for Round 1--
Abdelfattah has 163 International and 221 TG-Eligible, 384 total applications
Aizenman has 163 International and 221 TG-Eligible, 384 total applications
Amin has 163 International and 221 TG-Eligible, 384 total applications
Berson has 163 International and 221 TG-Eligible, 384 total applications
Chirila has 163 International and 221 TG-Eligible, 384 total applications
Fallon has 163 International and 221 TG-Eligible, 384 total applications
Jaworski has 163 International and 221 TG-Eligible, 384 total applications
Lambert has 163 International and 221 TG-Eligible, 384 total applications
Mayoral has 163 International and 221 TG-Eligible, 384 total applications
Moore has 163 International and 221 TG-Eligible, 384 total applications
Nassar has 163 International and 221 TG-Eligible, 384 total applications
Paradiso has 163 International and 221 TG-Eligible, 384 total applications
Sheinberg has 163 International and 221 TG-Eligible, 384 total applications

## Section 1D: Merge completed score cards 

Run once you receive completed score cards, in advance of first admissions meeting.

In [ ]:
#Concatenate all reviewer score cards 

print(f"--Merging Scorecards for Round {review_round}--")
reviewers = rev_groups['Reviewers'] #IF DGS SUBMITS A SCORESHEET NEED TO ADD NAME TO REVIEWERS FOR MERGE

score_df = applicant_info.copy()

for reviewer in reviewers:
    #First review all Person Names and make sure they are in applicants
    df = pd.read_excel(f"{directory_path}{sep}{reviewer}_Scorecard - Round {review_round}.xlsx")
    df.rename(columns={'Score': f"{reviewer} Score"}, inplace=True) #rename Score column to include reviewer name
    score_df = pd.merge(score_df,df[[f"{reviewer} Score",'Person Name']], how = 'left',on = 'Person Name')
    
    #Check for name mismatch (in case faculty reviews additional application and enters name incorrectly)
    set1 = set(df['Person Name'].values)
    if set1.difference(set(applicants)):
        print(f"{reviewer} name error")
        print(set1.difference(set(applicants)))
        
score_df.head()

#Reorder columns so scores are first
num_reviewers = len(reviewers)
cols = score_df.columns.tolist()
cols = cols[-num_reviewers:]+cols[:-num_reviewers]
score_df = score_df[cols]
score_df.head()

#Add mean score (should this be a formula instead in case things change??? maybe)
#mean_score = score_df.iloc[:,:num_reviewers].mean(axis = 1)
#sd_score = score_df.iloc[:,:num_reviewers].std(axis = 1)
score_df.insert(num_reviewers,'Mean','')
score_df.insert(num_reviewers+1,'SD','')
score_df.insert(num_reviewers+2,'Next Round? (y/n)','')
score_df.head()

#Separate tabs for International and TG-Eligible

score_int = score_df.loc[score_df['Citizenship Status'] == 'Foreign Citizen']
#score_int = score_int.sort_values(by='Mean')

score_tg = score_df.loc[score_df['Citizenship Status'] != 'Foreign Citizen']
#score_tg = score_tg.sort_values(by='Mean')


In [ ]:
print(f"--Formatting Merged Score Spreadsheet for Round {review_round}--")

#Create file, separate tabs for International and TG-Eligible pools
filename = f"Round {review_round} Scores.xlsx"
writer = pd.ExcelWriter(filename,engine = 'xlsxwriter')
#score_df.to_excel(writer,sheet_name = index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
score_int.to_excel(writer,sheet_name = 'International', index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
score_tg.to_excel(writer,sheet_name = 'TG-Eligible', index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format

# --Format score card headers--
workbook = writer.book
worksheet1 = writer.sheets['International']
worksheet2 = writer.sheets['TG-Eligible']

header_format = workbook.add_format({
    'bold': True,
    'font_size': 13,
    'align':'center',
    'font_color': '#ffffff',
    'fg_color':'#000000',
    'text_wrap':True
    
})
for col_num, value in enumerate(score_int.columns.values):
    worksheet1.write(0, col_num, value, header_format) #(row,column,value,format)


for col_num, value in enumerate(score_tg.columns.values):
    worksheet2.write(0, col_num, value, header_format) #(row,column,value,format)


# --Additional formatting--
first_row = 1
last_row = 300
first_col = score_int.columns.get_loc('Next Round? (y/n)') #column ID of Next Round?
last_col = score_int.columns.get_loc('Next Round? (y/n)') #column ID of Next Round?

#center align all of the scores
format0 = workbook.add_format({'align':'center'})
worksheet1.set_column(0,last_col,10,format0)
#worksheet1.set_column(last_col+1,25,25,10)
worksheet1.autofit() #Autofit (finicky, which is why I do minimum width first)


worksheet2.set_column(0,last_col,10,format0)
#worksheet2.set_column(last_col+1,25,25,10)
worksheet2.autofit() #Autofit (finicky, which is why I do minimum width first)
                      

# --Conditional formatting--

# Format Next 'Round? (y/n)'' column, green for y, red for n, yellow for m

nr_format1 = workbook.add_format({
    'bg_color':'#9BDE87'
})
nr_format2 = workbook.add_format({
    'bg_color':'#DE8E8E'
})
nr_format3 = workbook.add_format({
    'bg_color':'#EBE3AO'
})


worksheet1.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'m',
                             'format':nr_format3})
worksheet1.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'y',
                             'format':nr_format1})
worksheet1.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'n',
                             'format':nr_format2})

worksheet2.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'m',
                             'format':nr_format3})
worksheet2.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'y',
                             'format':nr_format1})
worksheet2.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'n',
                             'format':nr_format2})

# Color faculty scores (for prettiness! Definitely not necessary)
reviewer_colorspace = ['#F28FD4','#ffa3a2','#fcf6e7','#ff8e58','#da6175','#B05199','#446AB2',
                       '#3296A6','#24b8a0','#8ee6a1','#cae63c','#edd54f','#6bae36','#7da497','#c8cdbb']
for idx, value in enumerate(reviewers):
    rev_format = workbook.add_format({'bg_color':reviewer_colorspace[idx],
                                     'bold':True,
                                     'align':'center'})
    worksheet1.conditional_format(first_row,idx,last_row,idx,
                                 {'type':'no_blanks',
                                 'format':rev_format})
    worksheet2.conditional_format(first_row,idx,last_row,idx,
                                 {'type':'no_blanks',
                                 'format':rev_format})
    


writer.close() #Without this line the file will be corrupt



### Manual edits to spreadsheet
Open the spreadsheet and add the following by hand:
1. Count number of interviews (=COUNTIF(range,"=y") to bottom of 'Next Round (y/n)' column) for both sheets (probably a way to add this, but would likely not be any more efficient because need to hard code cell location)
2. Mean and SD score for each applicant (better to make it a formula in case corrections are made during the meeting?)
3. Mean score for each reviewer (better to make it a formula in case corrections are made during the meeting?)

# ROUND 2

## Section 2A: Import Data

In [ ]:
#TO-DO: Import multiple tabs and merge into one dataframe

#Import completed spreadsheet from above
review_round = 2
n_rev_per_app2 = 5 #number of reviewers assigned to each application
directory_path = f"Round {review_round}"
os.makedirs(directory_path, exist_ok=True)
print(f"Directory path = {directory_path}")

all_sheets = pd.read_excel(f"Round {review_round-1} Scores.xlsx", sheet_name=None)
applicant_info_r2 = pd.DataFrame()
for sheet_name, df in all_sheets.items(): #Concatenate info from all tabs into one (all have common)
    df['source sheet'] = sheet_name
    applicant_info_r2 = pd.concat([applicant_info_r2,df],axis = 0, ignore_index = True)

#Edit to get similar dataframe as for round 1
reviewers = rev_groups['Reviewers']
applicant_info_r2.drop(applicant_info_r2.iloc[:,0:len(reviewers)+2],axis = 1, inplace = True) #Drop reviewer scores, mean, and sd
applicant_info_r2 = applicant_info_r2.loc[applicant_info_r2['Next Round? (y/n)']=='y'] #Filter out those not going to next round
applicant_info_r2.drop(columns = 'Next Round? (y/n)',axis = 1, inplace = True) 
applicant_info_r2.rename(columns={'source sheet':'Pool'},inplace = True)



# All applicants in single pool
applicants_r2 = applicant_info_r2['Person Name'].values

# Separate TG-Eligible and International applicant pools
applicant_pools_r2 = {'International':applicant_info_r2.loc[applicant_info_r2['Citizenship Status']=='Foreign Citizen','Person Name'].values,
                  'TG-Eligible':applicant_info_r2.loc[applicant_info_r2['Citizenship Status']!='Foreign Citizen','Person Name'].values}


print(f"--Imported Data for Round {review_round}, {len(applicant_info_r2)} applications to review--")
print(len(applicant_info_r2))
applicant_info_r2.head() #First column should be Person Name


## Section 2B: Assign Reviewers

In [ ]:
# AT LEAST 2 SYS/COG and 2 CELL/MOLEC
start_time = time.time()
timeout_seconds = 3

print(f"--Assigning Reviewers for Round {review_round}--")
while True:
    current_time = time.time()
    if current_time - start_time >= timeout_seconds:
        print("Timeout reached, try again")
        break
    try:
        assignment_list, reviewer_workload = assign_reviewers_multipool(applicant_pools_r2,rev_groups,n_rev_per_app2) #If multiple applicant pools
        #assignment_list, reviewer_workload = assign_reviewers(applicants_r2,rev_groups,n_rev_per_app2) #If single pool
        break
    except ValueError as e:
        print(f"Error: {e}")

## Section 2C: Generate Scorecards

In [ ]:
#Generate reviewer score cards WITH formatting
print(f"--Generating Scorecards for Round {review_round}--")
reviewers = rev_groups['Reviewers']
for reviewer in reviewers:
    reviewer_df = applicant_info_r2[applicant_info_r2['Person Name'].isin(reviewer_workload['Applicants'].loc[reviewer_workload['Reviewer']==reviewer].values[0])]
    reviewer_df = reviewer_df.sort_values(by='Person Name')
    reviewer_df.insert(0, 'Score', 1) #Add blank column for Score (position, name, value)
    reviewer_df.insert(1, 'Notes', '') #Add blank column for Score (position, name, value)
    #reviewer_df = scorecard_base_df[scorecard_base_df['Reviewers'].isin([reviewer])].copy()
    #reviewer_df = reviewer_df.drop(columns = 'Reviewers')

    filename = f"{reviewer}_Scorecard - Round {review_round}.xlsx"
    writer = pd.ExcelWriter(filename,engine = 'xlsxwriter')
    reviewer_df.to_excel(writer,index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
    
    #Format score card
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    header_format = workbook.add_format({
        'bold': True,
        'font_size': 13,
        'align':'center',
        'font_color': '#ffffff',
        'fg_color':'#000000'       
        
    })
    for col_num, value in enumerate(reviewer_df.columns.values):
        worksheet.write(0, col_num, value, header_format) #(row,column,value,format)
    worksheet.set_column(0,25,15) #Set minimum column width
    worksheet.autofit() #Autofit (finicky, which is why I do minimum width first)

    writer.close() #Without this line the file will be corrupt

    #Print assignment distributions to review (sanity check)
    num_int = len(reviewer_df.loc[reviewer_df['Citizenship Status']=='Foreign Citizen'])
    num_tg = len(reviewer_df.loc[reviewer_df['Citizenship Status']!='Foreign Citizen'])
    print(f"{reviewer} has {num_int} International and {num_tg} TG-Eligible, {num_int + num_tg} total applications")    

#Generate spreadsheet for tags

In [ ]:
#Create blank interest spreadsheet to complete and merge for round 2 review

interest_df = pd.DataFrame(applicants_r2,columns=['Person Name'])
interest_df.insert(1,'Interest - Program Tag','')

filename = 'Applicant_Interests.xlsx'
writer = pd.ExcelWriter(filename,engine = 'xlsxwriter')
interest_df.to_excel(writer,index = False) 

workbook = writer.book
worksheet = writer.sheets['Sheet1']
worksheet.data_validation(1,1,len(applicants_r2),1,{
    'validate':'list',
    'source':['Any','CellMol','SysCogComp','BMI only']
})
worksheet.set_column(0,2,20) #Set minimum column width
worksheet.autofit() #Autofit (finicky, which is why I do minimum width first)

writer.close() #Without this line the file will be corrupt

## Section 2D: Merge completed score cards

In [ ]:
#Concatenate all reviewer score cards 

print(f"--Merging Scorecards for Round {review_round}--")
reviewers = rev_groups['Reviewers'] #IF DGS SUBMITS A SCORESHEET NEED TO ADD NAME TO REVIEWERS FOR MERGE


score_df = applicant_info_r2.copy()

for reviewer in reviewers:
    #First review all Person Names and make sure they are in applicants
    df = pd.read_excel(f"{reviewer}_Scorecard - Round {review_round}.xlsx")
    df.rename(columns={'Score': f"{reviewer} Score"}, inplace=True) #rename Score column to include reviewer name
    score_df = pd.merge(score_df,df[[f"{reviewer} Score",'Person Name']], how = 'left',on = 'Person Name')

    #Check for name mismatch (in case faculty reviews additional application and enters name incorrectly)
    set1 = set(df['Person Name'].values)
    if set1.difference(set(applicants)):
        print(f"{reviewer} name error")
        print(set1.difference(set(applicants)))




#Reorder columns so scores are first
num_reviewers = len(reviewers)
cols = score_df.columns.tolist()
cols = cols[-num_reviewers:]+cols[:-num_reviewers]
score_df = score_df[cols]

#Import and insert interest tags
df = pd.read_excel('Applicant_Interests.xlsx')
score_df = pd.merge(score_df,df,how = 'left', on = 'Person Name')
col_to_move = score_df.pop('Interest - Program Tag')
score_df.insert(num_reviewers+8,'Interest - Program Tag',col_to_move)


#Add mean score (should this be a formula instead in case things change??? maybe)
#mean_score = score_df.iloc[:,:num_reviewers].mean(axis = 1)
#sd_score = score_df.iloc[:,:num_reviewers].std(axis = 1)
score_df.insert(num_reviewers,'Mean','')
score_df.insert(num_reviewers+1,'SD','')
score_df.insert(num_reviewers+2,'Next Round? (y/n)','')

#Separate tabs for International and TG-Eligible

score_int = score_df.loc[score_df['Citizenship Status'] == 'Foreign Citizen']
score_int = score_int.sort_values(by='Person Name')

score_tg = score_df.loc[score_df['Citizenship Status'] != 'Foreign Citizen']
score_tg = score_tg.sort_values(by='Person Name')
score_df.head()

In [ ]:
print(f"--Formatting Merged Score Spreadsheet for Round {review_round}--")

#Create file, separate tabs for International and TG-Eligible pools
filename = f"Round {review_round} Scores.xlsx"
writer = pd.ExcelWriter(filename,engine = 'xlsxwriter')
#score_df.to_excel(writer,sheet_name = index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
score_int.to_excel(writer,sheet_name = 'International', index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
score_tg.to_excel(writer,sheet_name = 'TG-Eligible', index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format

# --Format score card headers--
workbook = writer.book
worksheet1 = writer.sheets['International']
worksheet2 = writer.sheets['TG-Eligible']

header_format = workbook.add_format({
    'bold': True,
    'font_size': 13,
    'align':'center',
    'font_color': '#ffffff',
    'fg_color':'#000000',
    'text_wrap':True
    
})
for col_num, value in enumerate(score_int.columns.values):
    worksheet1.write(0, col_num, value, header_format) #(row,column,value,format)


for col_num, value in enumerate(score_tg.columns.values):
    worksheet2.write(0, col_num, value, header_format) #(row,column,value,format)


# --Additional formatting--
first_row = 1
last_row = 300
first_col = score_int.columns.get_loc('Next Round? (y/n)') #column ID of Next Round?
last_col = score_int.columns.get_loc('Next Round? (y/n)') #column ID of Next Round?

#center align all of the scores
format0 = workbook.add_format({'align':'center'})
worksheet1.set_column(0,last_col,10,format0)
#worksheet1.set_column(last_col+1,25,25,10)
worksheet1.autofit() #Autofit (finicky, which is why I do minimum width first)


worksheet2.set_column(0,last_col,10,format0)
#worksheet2.set_column(last_col+1,25,25,10)
worksheet2.autofit() #Autofit (finicky, which is why I do minimum width first)
                      

# --Conditional formatting--

# Format Next 'Round? (y/n)'' column, green for y, red for n, yellow for m

nr_format1 = workbook.add_format({
    'bg_color':'#9BDE87'
})
nr_format2 = workbook.add_format({
    'bg_color':'#DE8E8E'
})
nr_format3 = workbook.add_format({
    'bg_color':'#EBE3AO'
})


worksheet1.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'m',
                             'format':nr_format3})
worksheet1.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'y',
                             'format':nr_format1})
worksheet1.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'n',
                             'format':nr_format2})

worksheet2.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'m',
                             'format':nr_format3})
worksheet2.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'y',
                             'format':nr_format1})
worksheet2.conditional_format(first_row, first_col, last_row, last_col,
                             {'type':'text',
                             'criteria':'containing',
                              'value':'n',
                             'format':nr_format2})

# Color faculty scores (for prettiness! Definitely not necessary)
reviewer_colorspace = ['#F28FD4','#ffa3a2','#fcf6e7','#ff8e58','#da6175','#B05199','#446AB2',
                       '#3296A6','#24b8a0','#8ee6a1','#edd54f','#6bae36','#7da497','#c8cdbb']
for idx, value in enumerate(reviewers):
    rev_format = workbook.add_format({'bg_color':reviewer_colorspace[idx],
                                     'bold':True,
                                     'align':'center'})
    worksheet1.conditional_format(first_row,idx,last_row,idx,
                                 {'type':'no_blanks',
                                 'format':rev_format})
    worksheet2.conditional_format(first_row,idx,last_row,idx,
                                 {'type':'no_blanks',
                                 'format':rev_format})
    


writer.close() #Without this line the file will be corrupt


### Manual edits to spreadsheet
Open the spreadsheet and add the following by hand:
1. Count number of interviews (=COUNTIF(range,"=y") to bottom of 'Next Round (y/n)' column) for both sheets (probably a way to add this, but would likely not be any more efficient because need to hard code cell location)
2. Mean and SD score for each applicant (better to make it a formula in case corrections are made during the meeting?)
3. Mean score for each reviewer (better to make it a formula in case corrections are made during the meeting?)

# Old Code
Might be useful until done troubleshooting?

In [6]:
#VERSION WITH SEPARATE APPLICANT POOLS - Fix uneven reviewer numbers
def assign_reviewers_multipool2(applicant_pools,rev_groups,n):
    # Args:
    #    applicant_pools: dictionary containing list of applicant names in each pool key
    #    rev_groups: dictionary categorizing reviewers into 'StudentReps', 'SysCog', 'CellMolec','Reviewers'
    #    n: single value, number of reviewers assigned to each application
    #
    # Returns:
    #    assignment_list_df: dataframe of each applicant and list of their reviewers
    #    reviewer_workload_df: dataframe of each reviewer and list of their assigned applicants

    reviewers = rev_groups['Reviewers'] #Unique list of reviewers
    pools = list(applicant_pools.keys())

    #Calculate workload, keeping total evenly distributed and with pool evenly distributed (ish)
    num_applicants = sum(len(value) for value in applicant_pools.values()) #Calculate total number of applicants
    workload_counts_total = calc_reviewer_workload(num_applicants,rev_groups,n) #Total reviews per reviewer   
    
    workload = {}
    for pool in pools[:-1]: #Calculate workloads independently for each pool except final pool
        workload[pool] = calc_reviewer_workload(len(applicant_pools[pool]),rev_groups,n)
            
    workload_pool_total = {}
    for inner_dict in workload.values(): #sum of workloads across pools, exculding final pool
        for key, value in inner_dict.items():
            workload_pool_total[key] = workload_pool_total.get(key,0) + value
       
    workload[pools[-1]] = {key: workload_counts_total[key] - workload_pool_total[key] for key in workload_counts_total}


    assignment_list = {}
    flag_count = 0
    for pool in applicant_pools:
        workload_counts = workload[pool]
        applicants = applicant_pools[pool]
       
        #Create assignment list
        random.shuffle(applicants)
        assignment_list[pool] = {applicant:[] for applicant in applicants}  
        for applicant in applicants:
            #Pull list of possible reviewers based on if their workload is already full or not
            available_reviewers = [candidates for candidates, count in workload_counts.items() if count > 0]
            if len(available_reviewers) < n:
                raise ValueError(
                    f"Cannot find {n} unique reviewers for '{applicant}'. The remaining pool "
                    f"of reviewers with available slots ({len(available_reviewers)}) is too small."
                )
                

            #Check if possible to select subset from available reviewers that will not break rules, else raise ValueError
            #From available reviewers, choose n at random    
            if sorted(available_reviewers) == sorted(rev_groups['StudentReps']):
                raise ValueError('Break Rule 1: Student only reviewers')

            if not any(item in available_reviewers for item in rev_groups['SysCog']) and any(item in available_reviewers for item in rev_groups['CellMolec']):
                flag = True
                flag_count = flag_count + 1
            else:
                flag = False
            #Draw random set of reviewers from those with workload available
            available_weights = [count for candidates, count in workload_counts.items() if count > 0]
            probabilities = [weight/sum(available_weights) for weight in available_weights]
            #chosen_reviewers = random.sample(available_reviewers, n)
            chosen_reviewers = np.random.choice(available_reviewers, n, replace = False,p=probabilities)  
            
            #Check reviewers chosen follows rule, or else pick another sample
            start_time = time.time()
            time_limit = 1
            
            if flag: #If there are no available reviewers from both reviewer pools, only concerned about Student Rep overlap
                while sorted(chosen_reviewers) == sorted(rev_groups['StudentReps']):
                    #chosen_reviewers = random.sample(available_reviewers, n) 
                    chosen_reviewers = np.random.choice(available_reviewers, n, replace = False,p=probabilities)  
            else: #If there are reviewers available from both reviewer pools, keep sampling until it follows rules            
            
                while sorted(chosen_reviewers) == sorted(rev_groups['StudentReps']) or not (any(item in chosen_reviewers for item in rev_groups['SysCog']) and any(item in chosen_reviewers for item in rev_groups['CellMolec'])):
                    #chosen_reviewers = random.sample(available_reviewers, n) 
                    chosen_reviewers = np.random.choice(available_reviewers, n, replace = False,p=probabilities)  
        
                    #Sometimes code gets hung. Introduce timeout to restart.
                    current_time = time.time()
                    if current_time - start_time >= time_limit:
                        raise ValueError("Assign Reviewers Timeout")
                        break
    
                
            assignment_list[pool][applicant] = chosen_reviewers
            for reviewer in chosen_reviewers:
                workload_counts[reviewer] -= 1

    merged_assignment_list = {key:value for sub_dict in assignment_list.values() for key,value in sub_dict.items()}

    #Reverse map assignments
    reviewer_workload = {reviewer:[] for reviewer in reviewers}   
    for app, revs in merged_assignment_list.items():
        for rev in revs:
            reviewer_workload[rev].append(app)

    # total = 0
    # for key,value in reviewer_workload.items():
    #     total = total + len(value)
    #     print(f"Number of apps for {key} is {len(value)}")
    # print(f"Total reviews is {total}")
        
    reviewer_workload_df = pd.DataFrame(reviewer_workload.items(),columns = ['Reviewer','Applicants'])
    assignment_list_df = pd.DataFrame(merged_assignment_list.items(),columns = ['Applicant','Reviewers'])
            
    return assignment_list_df, reviewer_workload_df, flag_count

    

In [7]:
def optimize_assignment_runs(applicant_pools, rev_groups, n, num_runs=100):
    best_assignment_df = None
    best_workload_df = None
    min_flags = sum(len(value) for value in applicant_pools.values())

    print(f"Starting {num_runs} optimization runs...")

    for i in range(num_runs):
        try:
            # 1. Run the assignment
            assignment_df, workload_df, current_flags = assign_reviewers_multipool2(
                applicant_pools, rev_groups, n
            )

            # 2. Compare the result
            if current_flags == 0:
                min_flags = current_flags
                best_assignment_df = assignment_df
                best_workload_df = workload_df
                break
            
            elif current_flags < min_flags:
                min_flags = current_flags
                best_assignment_df = assignment_df
                best_workload_df = workload_df
                print(f"Run {i+1}/{num_runs}: **New best found** with {min_flags} True flags.")
            # elif current_flags == min_flags:
            #     # You can add logic here to choose one based on a tie-breaker (e.g., workload variance)
            #     pass

        except (ValueError, TimeoutError) as e:
            # If the assignment fails due to a constraint or timeout, skip this run.
            print(f"Run {i+1}/{num_runs}: **Failed** due to error: {e}")
            continue

    print("\n--- Optimization Complete ---")
    print(f"Best assignment achieved a minimum of **{min_flags}** True flags.")

    # Return the best results found
    return best_assignment_df, best_workload_df

In [ ]:
assignment_list, reviewer_workload = optimize_assignment_runs(applicant_pools,rev_groups,n_rev_per_app)

for index,row in reviewer_workload.iterrows():
    print(f"Number of apps for {row['Reviewer']} is {len(row['Applicants'])}")

In [ ]:
#Analyze distribution of fields for each application (not required, just to do a manual verification while debugging)
assignment_list.head()
for applicant in assignment_list['Applicant'].values:
    chosen_reviewers = assignment_list.loc[assignment_list['Applicant']==applicant,'Reviewers'].values[0]
    if any(item in chosen_reviewers for item in rev_groups['SysCog']) and any(item in chosen_reviewers for item in rev_groups['CellMolec']):
        assignment_list.loc[assignment_list['Applicant']==applicant,'Check'] = True

test = assignment_list['Check'].value_counts()
print(test)
#assignment_list.to_excel('Assignments for review.xlsx')

In [ ]:
#For score cards: Have ALL applicant info listed with column indicating which are "assigned" - separate tabs for international and tg-eligible
print(f"--Generating Scorecards for Round {review_round}--")

reviewers = rev_groups['Reviewers']
for reviewer in reviewers:
    reviewer_df = applicant_info[applicant_info['Person Name'].isin(reviewer_workload['Applicants'].loc[reviewer_workload['Reviewer']==reviewer].values[0])]
    reviewer_df = reviewer_df.sort_values(by='Person Name')
    reviewer_df.insert(0, 'Score', 1) #Add blank column for Score (position, name, value)
    reviewer_df.insert(1, 'Notes', '') #Add blank column for Score (position, name, value)
    #reviewer_df = scorecard_base_df[scorecard_base_df['Reviewers'].isin([reviewer])].copy()
    #reviewer_df = reviewer_df.drop(columns = 'Reviewers')

    filename = f"{directory_path}{sep}{reviewer}_Scorecard - Round {review_round}.xlsx"
    writer = pd.ExcelWriter(filename,engine = 'xlsxwriter')
    reviewer_df.to_excel(writer,index = False,header=False) #header = false, Insert header below with xlsxwriter in order to format
    
    #Format score card
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    header_format = workbook.add_format({
        'bold': True,
        'font_size': 13,
        'align':'center',
        'font_color': '#ffffff',
        'fg_color':'#000000'       
        
    })
    for col_num, value in enumerate(reviewer_df.columns.values):
        worksheet.write(0, col_num, value, header_format) #(row,column,value,format)
    worksheet.set_column(0,25,15) #Set minimum column width
    worksheet.autofit() #Autofit (finicky, which is why I do minimum width first)

    writer.close() #Without this line the file will be corrupt

    num_int = len(reviewer_df.loc[reviewer_df['Citizenship Status']=='Foreign Citizen'])
    num_tg = len(reviewer_df.loc[reviewer_df['Citizenship Status']!='Foreign Citizen'])
    print(f"{reviewer} has {num_int} International and {num_tg} TG-Eligible, {num_int + num_tg} total applications")    
    
    
    